# LangChain Quickstart Follow-up

This notebook is a follow up on LangChain Quickstart Guide, which is available in its [documentation](https://python.langchain.com/en/latest/getting_started/getting_started.html). It includes additional notes for further understanding.

## 1. Get predictions from a Language Model

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../.env'))

True

In [2]:
# Check if environment variables have been loaded
import os

os.environ['TEST_VAR']

'Found'

In [5]:
from langchain.llms import OpenAI

# Temperature governs the randomness and thus the creativity of the responses.
llm = OpenAI(temperature=0.9)

text = "What is two plus two times nine equals?"
print(llm(text))



Two plus two times nine equals 36.


## 2. Manage Prompts for LLMs

This is kinda like defining and replacing a variable in a text, for example:

```python
thing = "line chart"
print(f'What is a {thing}')
```

In [9]:
from langchain.prompts import PromptTemplate

data_science_prompt = PromptTemplate(
    input_variables=["thing"],
    template="Explain {thing} in data science."
)

print(data_science_prompt.format(thing="A/B test"))
print(data_science_prompt.format(thing="Random Forest"))
print(data_science_prompt.format(thing="Large Language Model"))

Explain A/B test in data science.
Explain Random Forest in data science.
Explain Large Language Model in data science.


In [10]:
for thing in ["A/B test", "Random Forest", "Large Language Model"]:
    formatted_prompt = data_science_prompt.format(thing=thing)
    print(formatted_prompt)
    print(llm(formatted_prompt))
    print('-' * 50)

Explain A/B test in data science.


A/B testing is a method of data science used to compare two different versions of a product or feature in order to determine which one is more effective. In an A/B test, a group of users is randomly divided into two groups, each of which is given a different version of the product or feature to use. The success of one version is then measured against the other in order to determine which is more effective. A/B tests are used to optimize a product for maximum impact, and are widely used in online marketing, website design, product development, and more.
--------------------------------------------------
Explain Random Forest in data science.


Random Forest is an ensemble learning method used for classification and regression that operates by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random Forest leverag

## 3. Chains: Combine LLMs and prompts in multi-step workflows

We could have shorten the code in section 2 by using chains as follows:

In [11]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=data_science_prompt)

In [13]:
for thing in ["A/B test", "Random Forest", "Large Language Model"]:
    print(thing)
    print(chain.run(thing))
    print('-' * 50)

A/B test


A/B testing, also known as split testing, is a statistical method that allows data scientists to compare two versions of a web page, application, or product with the goal of determining which one performs better. It is used to determine which version will have the greatest impact on user behavior and provide the most benefits. In an A/B test, a single variable (the "A" or the "B") is tested to determine which performs better across a variety of metrics. For example, a company may test two versions of a landing page to see which one produces more conversions.
--------------------------------------------------
Random Forest


Random Forest is an ensemble method for classifying and predicting data in machine learning. It creates a group of decision trees (an ensemble) from a randomly selected subset of training data. Each tree in the ensemble is trained on a different subset of features and then makes predictions independently. The final prediction is determined by a majority v

## 4. Dynamically Call Chains Based on User Input

Contexts for agents:
- [Tool](https://python.langchain.com/en/latest/modules/agents/tools/getting_started.html): A function that performs a specific duty, note that some require an attached API key. Some available tools include:
  - `python_repl`: Python shell for executing python commands.
  - `serpapi`: Search engine, useful for when you need to answer questions about current events.
  - `wolfram-alpha`: Wolfram alpha search engine for mathematics, science, technology, cultural, social questions.
  - `requests`: Python requests module, get a specific URL as input, and output will be text on that page.
  - `terminal`: Execute commands in a terminal.
  - `llm-math`: Useful when need to answer questions about math.
  - `wikipedia`: Search on Wikipedia.
- LLM: The language model powering the agent.
- Agent: The agent to use. It can be a built-in or customized agent. This notebook include an example of a built-in agent.

In [15]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.llms import OpenAI

# Load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Load tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Initialize the agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [18]:
# Test a query
query = "What is the maximum temperature in Hanoi yesterday in Celsius? Is it greater than 30 degree."
agent.run(query)



> Entering new AgentExecutor chain...
 I need to find out the temperature in Hanoi yesterday
Action: Search
Action Input: "maximum temperature in Hanoi yesterday"
Observation: Hanoi Temperature Yesterday. Maximum temperature yesterday: 102 °F (at 2:30 pm) Minimum temperature yesterday: 79 °F (at 1:30 am) Average temperature ...
Thought: I need to convert the temperature from Fahrenheit to Celsius
Action: Calculator
Action Input: 102 °F to Celsius
Observation: Answer: 38.888888888888886
Thought: I now know the final answer
Final Answer: The maximum temperature in Hanoi yesterday in Celsius was 38.888888888888886 degrees, which is greater than 30 degrees.

> Finished chain.


'The maximum temperature in Hanoi yesterday in Celsius was 38.888888888888886 degrees, which is greater than 30 degrees.'

The Agent type decides which actions to taken and in what order. Options can be found [here](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html).
- `zero-shot-react-description`: Use ReAct framework to determine which tool to use based solely on the tool's description.
- `react-docstore`: Interact with a docstore, using 2 tools, a `Search` tool and a `Lookup` tool.
- `self-ask-with-search`: Lookup factual answers to questions.
- `conversational-react-description`: Designed to make the agent helpful and conversational.

In [34]:
# All posible AgentType
AgentType._member_names_

['ZERO_SHOT_REACT_DESCRIPTION',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION']

## 5. Memory: Add State to Chains and Agents

In [19]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

In [20]:
output = conversation.predict(input="Hi colleague")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi colleague
AI:

> Finished chain.
 Hi there! It's nice to meet you. I'm an AI created to help you with your work. What can I do for you today?


In [22]:
print(conversation.predict(input="I am having a problem with my project. Can you help me?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi colleague
AI:  Hi there! It's nice to meet you. I'm an AI created to help you with your work. What can I do for you today?
Human: I am having a problem with my project. Can you help me?
AI:

> Finished chain.
 Absolutely! What kind of problem are you having?


In [23]:
print(conversation.predict(input="What is LangChain and possible usecases to deploy it in production?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi colleague
AI:  Hi there! It's nice to meet you. I'm an AI created to help you with your work. What can I do for you today?
Human: I am having a problem with my project. Can you help me?
AI:  Absolutely! What kind of problem are you having?
Human: What is LangChain and possible usecases to deploy it in production?
AI:

> Finished chain.
 LangChain is a blockchain-based platform that enables developers to create decentralized applications (dApps) that are language-agnostic. It allows developers to create applications that can be used across multiple languages, making it easier to develop and deploy applications in production. Possible use cases for LangChain in

In [24]:
print(conversation.predict(input="It seems weird, I don't see it as a blockchain-based platform, I am using it to chat to you"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi colleague
AI:  Hi there! It's nice to meet you. I'm an AI created to help you with your work. What can I do for you today?
Human: I am having a problem with my project. Can you help me?
AI:  Absolutely! What kind of problem are you having?
Human: What is LangChain and possible usecases to deploy it in production?
AI:  LangChain is a blockchain-based platform that enables developers to create decentralized applications (dApps) that are language-agnostic. It allows developers to create applications that can be used across multiple languages, making it easier to develop and deploy applications in production. Possible use cases for LangChain include creating mult

I believe that LangChain has been recently released, therefore LLM does not know about it.

## 6. Chat Prompt Templates

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Initialize the LLM Chat Model
chat = ChatOpenAI(temperature=0)

# Initialize the templates
template = "You are a senior data scientist who are trying to explain a concept in {language} for a person at {level} level"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [31]:
# Run the chain
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(
    language="Vietnamese",
    level="5-year-old",
    text="What is one hot encoding?"
)

'One hot encoding là một kỹ thuật được sử dụng trong machine learning để biến đổi các giá trị của một biến thành các vector nhị phân. Ví dụ, nếu chúng ta có một biến "màu sắc" với các giá trị "đỏ", "xanh lá cây" và "xanh dương", chúng ta có thể sử dụng one hot encoding để biến đổi các giá trị này thành các vector nhị phân. Vector cho giá trị "đỏ" sẽ có giá trị 1 tại vị trí đầu tiên và giá trị 0 tại các vị trí còn lại, vector cho giá trị "xanh lá cây" sẽ có giá trị 1 tại vị trí thứ hai và giá trị 0 tại các vị trí còn lại, và vector cho giá trị "xanh dương" sẽ có giá trị 1 tại vị trí thứ ba và giá trị 0 tại các vị trí còn lại. Kỹ thuật này giúp cho các giá trị của biến có thể được sử dụng trong các mô hình machine learning một cách dễ dàng và hiệu quả.'

## 7. Agents with Chat Models

In [39]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: I need to use a search engine to find Olivia Wilde's boyfriend and a calculator to raise his age to the 0.23 power.
Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```

Observation: Looks like Olivia Wilde and Jason Sudeikis are starting 2023 on good terms. Amid their highly publicized custody battle – and the actress' ...
Thought:Now I need to use a calculator to raise Jason Sudeikis' age to the 0.23 power.
Action:
```
{
  "action": "Calculator",
  "action_input": "47^0.23"
}
```


Observation: Answer: 2.4242784855673896
Thought:I now know that Jason Sudeikis' current age raised to the 0.23 power is 2.4242784855673896.
Final Answer: Jason Sudeikis' current age raised to the 0.23 power is 2.4242784855673896.

> Finished chain.


"Jason Sudeikis' current age raised to the 0.23 power is 2.4242784855673896."

## 8. Memory on Chat Agents

In [38]:
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

print(conversation.predict(input="Hi there!"))
print(conversation.predict(input="I'm doing well! Just having a conversation with an AI."))
print(conversation.predict(input="Tell me about yourself."))

Hello! How can I assist you today?
That sounds like fun! I'm happy to chat with you. What would you like to talk about?
Sure! I am an AI language model created by OpenAI. I was trained on a large dataset of text from the internet, which allows me to understand and generate human-like language. I can answer questions, provide information, and even have conversations with people like you! Is there anything specific you'd like to know about me?
